In [1]:
pip install lightgbm optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 20.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score, cohen_kappa_score, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier, LGBMRegressor
# from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/TS-S3-Ep5/train.csv'
file_key_2 = 'Tabular-Playground-Series/TS-S3-Ep5/test.csv'
file_key_3 = 'Tabular-Playground-Series/TS-S3-Ep5/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

## Enginering features
train['alcohol_density'] = train['alcohol'] * train['density']
train['sulphate/density'] = train['sulphates']  / train['density']
train['alcohol_sulphate'] = train['alcohol'] * train['sulphates']

test['alcohol_density'] = test['alcohol']  * test['density']
test['sulphate/density'] = test['sulphates']  / test['density']
test['alcohol_sulphate'] = test['alcohol'] * test['sulphates']

test_md = test.copy()

X = train[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]
Y = train['quality'] 

test_md = test_md[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]

# Optimal Rounder

In [2]:
from functools import partial
import numpy as np
import scipy as sp

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8

        ll = cohen_kappa_score(y, X_p, weights = 'quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X = X, y = y)
        initial_coef = [3.5, 4.5, 5.5, 6.5, 7.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8
        return X_p

    def coefficients(self):
        return self.coef_['x']

# Optuna Optimzation

In [17]:
def objective(trial):
    
    ## Parameters to be evaluated
    param = dict(objective = 'regression_l1',
                 verbose = -100,
                 boosting_type = 'gbdt', 
                 random_state = 906,
                 n_estimators = trial.suggest_int('n_estimators', 300, 10000),
                 learning_rate = trial.suggest_float('learning_rate', 0.001, 1, log = True),
                 max_depth = trial.suggest_int('max_depth', 3, 12),
                 lambda_l1 = trial.suggest_float('lambda_l1', 0.01, 10.0, log = True),
                 lambda_l2 = trial.suggest_float('lambda_l2', 0.01, 10.0, log = True),
                 num_leaves = trial.suggest_int('num_leaves', 2, 100),
                 bagging_fraction = trial.suggest_float('bagging_fraction', 0.2, 0.9),
                 feature_fraction = trial.suggest_float('feature_fraction', 0.2, 0.9)
                 )

    
    scores = []

#     skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 906)
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        
        print(fold, end = ' ')
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train , y_valid = Y.iloc[train_idx] , Y.iloc[valid_idx]

        model = LGBMRegressor(**param)
        model.fit(X_train, y_train)
        
        preds_train = model.predict(X_train)
        preds_valid = model.predict(X_valid)
        
        optR = OptimizedRounder()
        optR.fit(preds_train, y_train)
        coef = optR.coefficients()
        preds_valid = optR.predict(preds_valid, coef).astype(int)
        
        score = cohen_kappa_score(y_valid,  preds_valid, weights = "quadratic")
        scores.append(score)
        
    return np.mean(scores)

In [18]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 50, timeout = 3600)

[I 2023-02-08 15:29:46,731] A new study created in memory with name: no-name-15e96651-4389-4e22-8513-a2bc1d95d5ca


0 [LightGBM] [Warning] lambda_l1 is set=0.4611750875550995, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.4611750875550995
[LightGBM] [Warning] bagging_fraction is set=0.4347627657334713, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4347627657334713
[LightGBM] [Warning] feature_fraction is set=0.655767761194281, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.655767761194281
[LightGBM] [Warning] lambda_l2 is set=1.5831600435314832, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5831600435314832
1 [LightGBM] [Warning] lambda_l1 is set=0.4611750875550995, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.4611750875550995
[LightGBM] [Warning] bagging_fraction is set=0.4347627657334713, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4347627657334713
[LightGBM] [Warning] feature_fraction is set=0.655767761194281, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.65576776119428

[I 2023-02-08 15:30:03,698] Trial 0 finished with value: 0.516689131644715 and parameters: {'n_estimators': 3373, 'learning_rate': 0.16739079532185688, 'max_depth': 6, 'lambda_l1': 0.4611750875550995, 'lambda_l2': 1.5831600435314832, 'num_leaves': 14, 'bagging_fraction': 0.4347627657334713, 'feature_fraction': 0.655767761194281}. Best is trial 0 with value: 0.516689131644715.


0 [LightGBM] [Warning] lambda_l1 is set=0.02534001537032737, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.02534001537032737
[LightGBM] [Warning] bagging_fraction is set=0.7361433861295443, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7361433861295443
[LightGBM] [Warning] feature_fraction is set=0.8264289814555326, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8264289814555326
[LightGBM] [Warning] lambda_l2 is set=5.342166436108739, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.342166436108739
1 [LightGBM] [Warning] lambda_l1 is set=0.02534001537032737, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.02534001537032737
[LightGBM] [Warning] bagging_fraction is set=0.7361433861295443, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7361433861295443
[LightGBM] [Warning] feature_fraction is set=0.8264289814555326, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.826428981

[I 2023-02-08 15:30:12,212] Trial 1 finished with value: 0.5241736422613635 and parameters: {'n_estimators': 716, 'learning_rate': 0.0925117868493144, 'max_depth': 5, 'lambda_l1': 0.02534001537032737, 'lambda_l2': 5.342166436108739, 'num_leaves': 90, 'bagging_fraction': 0.7361433861295443, 'feature_fraction': 0.8264289814555326}. Best is trial 1 with value: 0.5241736422613635.


...

[I 2023-02-08 15:43:36,266] Trial 47 finished with value: 0.5447255518379654 and parameters: {'n_estimators': 357, 'learning_rate': 0.007513603315844528, 'max_depth': 3, 'lambda_l1': 0.04040507332550002, 'lambda_l2': 1.0923461498341436, 'num_leaves': 6, 'bagging_fraction': 0.6021446672385868, 'feature_fraction': 0.27647534931977313}. Best is trial 35 with value: 0.5576314002889131.


0 [LightGBM] [Warning] lambda_l1 is set=0.018508053970504694, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.018508053970504694
[LightGBM] [Warning] bagging_fraction is set=0.5879710299266914, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5879710299266914
[LightGBM] [Warning] feature_fraction is set=0.32334467410853635, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.32334467410853635
[LightGBM] [Warning] lambda_l2 is set=2.737264589384271, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.737264589384271
1 [LightGBM] [Warning] lambda_l1 is set=0.018508053970504694, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.018508053970504694
[LightGBM] [Warning] bagging_fraction is set=0.5879710299266914, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5879710299266914
[LightGBM] [Warning] feature_fraction is set=0.32334467410853635, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.32

[I 2023-02-08 15:43:43,237] Trial 48 finished with value: 0.5395209142722307 and parameters: {'n_estimators': 778, 'learning_rate': 0.012838308749953665, 'max_depth': 4, 'lambda_l1': 0.018508053970504694, 'lambda_l2': 2.737264589384271, 'num_leaves': 13, 'bagging_fraction': 0.5879710299266914, 'feature_fraction': 0.32334467410853635}. Best is trial 35 with value: 0.5576314002889131.


0 [LightGBM] [Warning] lambda_l1 is set=0.03339064349263303, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03339064349263303
[LightGBM] [Warning] bagging_fraction is set=0.6084339432148098, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6084339432148098
[LightGBM] [Warning] feature_fraction is set=0.23284683514769403, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.23284683514769403
[LightGBM] [Warning] lambda_l2 is set=1.187247856328268, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.187247856328268
1 [LightGBM] [Warning] lambda_l1 is set=0.03339064349263303, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.03339064349263303
[LightGBM] [Warning] bagging_fraction is set=0.6084339432148098, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6084339432148098
[LightGBM] [Warning] feature_fraction is set=0.23284683514769403, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.232846

[I 2023-02-08 15:43:51,925] Trial 49 finished with value: 0.5420252119823579 and parameters: {'n_estimators': 2499, 'learning_rate': 0.0048422561879380545, 'max_depth': 5, 'lambda_l1': 0.03339064349263303, 'lambda_l2': 1.187247856328268, 'num_leaves': 9, 'bagging_fraction': 0.6084339432148098, 'feature_fraction': 0.23284683514769403}. Best is trial 35 with value: 0.5576314002889131.


In [22]:
study.best_trial

FrozenTrial(number=35, state=TrialState.COMPLETE, values=[0.5576314002889131], datetime_start=datetime.datetime(2023, 2, 8, 15, 41, 31, 10888), datetime_complete=datetime.datetime(2023, 2, 8, 15, 41, 44, 292933), params={'n_estimators': 2057, 'learning_rate': 0.0028340232667834744, 'max_depth': 6, 'lambda_l1': 0.18078853991285831, 'lambda_l2': 0.7550241946424767, 'num_leaves': 82, 'bagging_fraction': 0.37125129346914726, 'feature_fraction': 0.3773044655610498}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=10000, log=False, low=300, step=1), 'learning_rate': FloatDistribution(high=1.0, log=True, low=0.001, step=None), 'max_depth': IntDistribution(high=12, log=False, low=3, step=1), 'lambda_l1': FloatDistribution(high=10.0, log=True, low=0.01, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=0.01, step=None), 'num_leaves': IntDistribution(high=100, log=False, low=2, step=1), 'bagging_fraction': FloatDistrib

In [20]:
study.best_trial.values

[0.5576314002889131]

In [23]:
study.best_trial.params

{'n_estimators': 2057,
 'learning_rate': 0.0028340232667834744,
 'max_depth': 6,
 'lambda_l1': 0.18078853991285831,
 'lambda_l2': 0.7550241946424767,
 'num_leaves': 82,
 'bagging_fraction': 0.37125129346914726,
 'feature_fraction': 0.3773044655610498}

# LightGBM Modeling

In [24]:
lgb_cv_scores, lgb_imp = list(), list()
preds = list()

skf = KFold(n_splits = 5, shuffle = True, random_state = 906)
    
for train_ix, test_ix in skf.split(X, Y):
        
    ## Splitting the data 
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    
    ## Building the model
    lgb_md = LGBMRegressor(n_estimators = 2057,
                            max_depth = 6,
                            learning_rate = 0.0028340232667834744,
                            num_leaves = 82,
                            lambda_l1 = 0.18078853991285831,
                            lambda_l2 = 0.7550241946424767,
                            bagging_fraction = 0.37125129346914726,
                            feature_fraction = 0.3773044655610498,
                            random_state = 906).fit(X_train, Y_train)
    lgb_imp.append(lgb_md.feature_importances_)
    
    ## Predicting on X_test and test
    lgb_pred_1 = lgb_md.predict(X_test)
    lgb_pred_2 = lgb_md.predict(test_md)
        
    ## Applying Optimal Rounder (using abhishek approach)
    optR = OptimizedRounder()
    optR.fit(lgb_md.predict(X_train), Y_train)
    coef = optR.coefficients()
    lgb_pred_1 = optR.predict(lgb_pred_1, coef).astype(int)
    lgb_pred_2 = optR.predict(lgb_pred_2, coef).astype(int)
        
    ## Computing roc-auc score
    lgb_cv_scores.append(cohen_kappa_score(Y_test, lgb_pred_1, weights = 'quadratic'))
    preds.append(lgb_pred_2)

lgb_cv_score = np.mean(lgb_cv_scores)    
print('The average weighted kappa score score over 5-folds (run 5 times) is:', lgb_cv_score)

[LightGBM] [Warning] lambda_l1 is set=0.18078853991285831, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.18078853991285831
[LightGBM] [Warning] bagging_fraction is set=0.37125129346914726, subsample=1.0 will be ignored. Current value: bagging_fraction=0.37125129346914726
[LightGBM] [Warning] feature_fraction is set=0.3773044655610498, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3773044655610498
[LightGBM] [Warning] lambda_l2 is set=0.7550241946424767, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7550241946424767
[LightGBM] [Warning] lambda_l1 is set=0.18078853991285831, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.18078853991285831
[LightGBM] [Warning] bagging_fraction is set=0.37125129346914726, subsample=1.0 will be ignored. Current value: bagging_fraction=0.37125129346914726
[LightGBM] [Warning] feature_fraction is set=0.3773044655610498, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3773044

In [25]:
lgb_preds_test = pd.DataFrame(preds).mode(axis = 0).loc[0, ]

submission['quality'] = lgb_preds_test.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,5
3,2059,6
4,2060,7


In [26]:
submission['quality'].value_counts()

5    576
6    431
7    365
Name: quality, dtype: int64

In [49]:
lgb_md = LGBMRegressor(n_estimators = 2057,
                            max_depth = 6,
                            learning_rate = 0.0028340232667834744,
                            num_leaves = 82,
                            lambda_l1 = 0.18078853991285831,
                            lambda_l2 = 0.7550241946424767,
                            bagging_fraction = 0.37125129346914726,
                            feature_fraction = 0.3773044655610498,
                            random_state = 906).fit(X, Y)

optR = OptimizedRounder()
optR.fit(lgb_md.predict(X), Y)
coef = optR.coefficients()
lgb_pred = lgb_md.predict(test_md)
lgb_pred = optR.predict(lgb_pred, coef).astype(int)

[LightGBM] [Warning] lambda_l1 is set=0.18078853991285831, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.18078853991285831
[LightGBM] [Warning] bagging_fraction is set=0.37125129346914726, subsample=1.0 will be ignored. Current value: bagging_fraction=0.37125129346914726
[LightGBM] [Warning] feature_fraction is set=0.3773044655610498, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3773044655610498
[LightGBM] [Warning] lambda_l2 is set=0.7550241946424767, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7550241946424767


In [57]:
coef

array([3.35384325, 5.0094123 , 5.64198756, 6.14471119, 7.58699092])

In [58]:
submission['quality'] = lgb_pred
submission.head() 

,Id,quality
0,2056,5
1,2057,6
2,2058,5
3,2059,6
4,2060,6


In [59]:
submission['quality'].value_counts()

5    557
6    396
7    363
4     56
Name: quality, dtype: int64

In [60]:
submission.to_csv('LightGBM_Reg_full_FE_4.csv', index = False)

In [27]:
best_sub = pd.read_csv('kaggle_submission.csv')
best_sub['quality'].value_counts()

6    691
5    529
7    152
Name: quality, dtype: int64

In [28]:
oscar_best = pd.read_csv('XGB_Reg_FE_2.csv')
oscar_best['quality'].value_counts()

5    634
6    425
7    313
Name: quality, dtype: int64

In [ ]:
submission.to_csv('LightGBM_Reg_FE_3.csv', index = False)

In [29]:
2 + 2

4

In [30]:
data_out = pd.merge(best_sub, oscar_best, on = 'Id')
data_out.head(10)

,Id,quality_x,quality_y
0,2056,5,5
1,2057,6,6
2,2058,5,5
3,2059,6,6
4,2060,6,6
5,2061,6,6
6,2062,6,5
7,2063,6,6
8,2064,7,6
9,2065,6,6


In [39]:
to_check = data_out[~(data_out['quality_x'] == data_out['quality_y'])]['Id'].tolist()

In [40]:
train.describe()

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,alcohol_density,sulphate/density,alcohol_sulphate
count,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000,2056.000000
mean,1027.500000,8.365175,0.527601,0.265058,2.398881,0.081856,16.955982,49.236868,0.996748,3.310569,0.641308,10.414972,5.720817,10.380431,0.643404,6.714287
std,593.660397,1.705390,0.173164,0.188267,0.858824,0.023729,10.009710,32.961141,0.001827,0.142321,0.137942,1.028825,0.853146,1.018378,0.138401,1.736270
min,0.000000,5.000000,0.180000,0.000000,1.200000,0.012000,1.000000,7.000000,0.990070,2.740000,0.390000,8.700000,3.000000,8.667810,0.390938,3.783000
25%,513.750000,7.200000,0.390000,0.090000,1.900000,0.071000,8.000000,22.000000,0.995600,3.200000,0.550000,9.500000,5.000000,9.482900,0.550028,5.400000
50%,1027.500000,7.950000,0.520000,0.250000,2.200000,0.079000,16.000000,44.000000,0.996700,3.310000,0.610000,10.100000,6.000000,10.065660,0.611820,6.326000
75%,1541.250000,9.200000,0.640000,0.420000,2.600000,0.090000,24.000000,65.000000,0.997800,3.390000,0.720000,11.000000,6.000000,11.001925,0.719748,7.702000
max,2055.000000,15.900000,1.580000,0.760000,14.000000,0.414000,68.000000,289.000000,1.003690,3.780000,1.950000,14.000000,8.000000,13.962480,1.956260,21.450000


In [47]:
train[train['alcohol'] > 11]

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,alcohol_density,sulphate/density,alcohol_sulphate
0,0,8.0,0.50,0.39,2.2,0.073,30.0,39.0,0.99572,3.33,0.77,12.1,6,12.048212,0.773310,9.317
1,1,9.3,0.30,0.73,2.3,0.092,30.0,67.0,0.99854,3.32,0.67,12.8,6,12.781312,0.670980,8.576
2,2,7.1,0.51,0.03,2.1,0.059,3.0,12.0,0.99660,3.52,0.73,11.3,7,11.261580,0.732490,8.249
6,6,7.2,0.87,0.00,2.3,0.080,6.0,18.0,0.99552,3.34,0.60,11.3,6,11.249376,0.602700,6.780
16,16,11.5,0.27,0.60,2.3,0.089,11.0,25.0,0.99628,3.10,0.44,12.0,7,11.955360,0.441643,5.280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,2044,8.2,0.29,0.42,4.3,0.085,6.0,12.0,0.99627,3.30,0.62,12.1,6,12.054867,0.622321,7.502
2050,2050,11.9,0.38,0.49,2.3,0.081,24.0,42.0,0.99940,3.15,0.59,11.5,6,11.493100,0.590354,6.785
2051,2051,6.6,0.31,0.13,2.0,0.056,29.0,42.0,0.99388,3.52,0.87,12.0,7,11.926560,0.875357,10.440
2053,2053,7.7,0.43,0.42,1.7,0.071,19.0,37.0,0.99258,3.32,0.77,12.5,8,12.407250,0.775756,9.625


In [45]:
test[np.isin(test['Id'], to_check)].describe()

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,alcohol_density,sulphate/density,alcohol_sulphate
count,284.000000,284.000000,284.000000,284.00000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.000000
mean,2758.559859,8.634859,0.496232,0.29838,2.497007,0.082444,16.147887,42.376761,0.996756,3.302500,0.731268,10.549296,10.514598,0.733621,7.699254
std,399.555866,1.824010,0.177911,0.19309,0.942124,0.020255,9.157011,26.720295,0.001735,0.133514,0.136780,0.824771,0.815785,0.137098,1.469124
min,2062.000000,5.000000,0.200000,0.00000,1.500000,0.041000,3.000000,8.000000,0.991820,2.890000,0.450000,9.000000,8.969400,0.452261,4.950000
25%,2398.250000,7.300000,0.360000,0.12000,2.000000,0.069750,7.000000,22.000000,0.995600,3.200000,0.640000,10.000000,9.960400,0.644944,6.385000
50%,2785.500000,8.250000,0.455000,0.31000,2.200000,0.078000,15.000000,37.000000,0.996600,3.300000,0.730000,10.500000,10.470180,0.733510,7.910000
75%,3092.750000,9.825000,0.610000,0.45250,2.600000,0.089000,23.000000,54.000000,0.997885,3.382500,0.802500,11.000000,10.950555,0.805491,8.585000
max,3424.000000,15.600000,1.090000,0.74000,7.900000,0.214000,41.000000,147.000000,1.002890,3.780000,1.620000,14.000000,13.941200,1.625527,16.200000
